# Homework 3: Data visualization

This homework assignment is designed to give you practice manipulating and visualizing data. 

You will need to download the **LexicalData.csv** and **Items.csv** files from the *Homework/lexDat* folder in the class GitHub repository. 

This data is a subset of the [English Lexicon Project database](https://elexicon.wustl.edu/). It provides the reaction times (in milliseconds) of many subjects as they are presented with letter strings and asked to decide, as quickly and as accurately as possible, whether the letter string is a word or not. The **Items.csv** provides characteristics of the words used, namely frequency (how common is this word?) and length (how many letters?). Unlike in the previous homework, there isn't any missing data in the **LexicalData.csv** file. 

*Data courtesy of Balota, D.A., Yap, M.J., Cortese, M.J., Hutchison, K.A., Kessler, B., Loftis, B., Neely, J.H., Nelson, D.L., Simpson, G.B., & Treiman, R. (2007). The English Lexicon Project. Behavior Research Methods, 39, 445-459.*

---
## Loading the Data (1 point)

Use the `setwd` (or `system ("gdown")` if you're working in colab) and `read.csv` functions to load data from the **LexicalData.csv** and **Items.csv** files. Then, as in the previous homework, remove the commas from the reaction times and convert them from strings to numbers. Use `head` to look at the first few rows of each data frame. 

*Note: the `Freq_HAL` variable in **Items.csv** has a similar formatting issue, using string values with commas. We're not going to worry about fixing this since we're only using `Log_Freq_HAL`, which is the natural log transformation of `Freq_HAL`, in this homework.*

In [1]:
#INSERT CODE HERE

## Reformatting the Data (1 points)

First you'll need to load the `tidyverse` library.

In [2]:
#INSERT CODE HERE

 In order to get some insights about what characteristics of words make them easier to recognize, it would be useful to have reaction times and word characteristics in the same data frame. Add the `Length` and `Log_Freq_HAL` variables from the items data frame to the reaction times data frame using `left_join()`. You may want to use `select()` and the piping operator `%>%`, as well.

In [3]:
#INSERT CODE HERE

## Visualizing the Data (7 points)

Now, let's visualize some aspects of the data. For each of the following questions, use `ggplot2` to plot the data in a way that helps you answer the question. Then write a qualitative response to the question. **Keep in mind the 9 properties of graphical excellence as you make decisions for this section.**

*Question 1:* How are the reaction time values distributed?

In [4]:
#INSERT CODE HERE

> *Write your response here.*


*Question 2:* Is there a relationship between word length and log frequency? If so, how would you describe it? 

In [5]:
#INSERT CODE HERE

> *Write your response here.*


*Question 3:* What is the relationship between log word frequency and lexical decision reaction time? (hint: including `stat_summary` will be helpful)

In [6]:
#INSERT CODE HERE

> *Write your response here.*


*Question 4:* Is the overall relationship between log word frequency and lexical decision reaction time different for short words versus long words? (hint: try binning both length and frequency)

In [7]:
#INSERT CODE HERE

> *Write your response here.*

## Reflection (1 point)
How might you approach analyzing this data differently after the insights you gained from these visualizations? 

> *Write your response here.* 

**DUE:** 5pm EST, March 4, 2021

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here. 
> *Someone's Name*